In [14]:
from copy import deepcopy
import yaml
from pathlib import Path
import torch
import shutil
import matplotlib.pyplot as plt

%reload_ext autoreload
%autoreload 2

# Cut boxes

In [24]:
dataset_name = "benchmark_dataset_2d_100datapoints inputs_pksi"
with open("paths.yaml", "r") as paths:
    paths = yaml.safe_load(paths)
    prepared1_dir = Path(paths["datasets_prepared_dir"]) / dataset_name

## Cut dataset into x boxes

In [32]:
number_boxes = 16
number_datapoints = 2
prepared_pieces_dir = Path(paths["datasets_prepared_dir"]) / f"{dataset_name} cut_{number_boxes}pieces separate_boxes"
prepared_pieces_dir.mkdir(parents=True, exist_ok=True)
for box in range(number_boxes):
    (prepared_pieces_dir / f"Inputs Box {box}").mkdir(parents=True, exist_ok=True)
    (prepared_pieces_dir / f"Label Box {box}").mkdir(parents=True, exist_ok=True)
shutil.copy(prepared1_dir / "info.yaml", prepared_pieces_dir / "info.yaml")

j = 0
for datapoint in zip((prepared1_dir / "Inputs").iterdir(), (prepared1_dir / "Labels").iterdir()):
    input = torch.load(datapoint[0])
    label = torch.load(datapoint[1])
    name = datapoint[0].stem

    input_boxes = []
    label_boxes = []
    for i in range(number_boxes):
        len_box = input.shape[1] // number_boxes
        input_boxes.append(input[:, i * len_box : (i + 1) * len_box, :])
        label_boxes.append(label[:, i * len_box : (i + 1) * len_box, :])


    for i in range(number_boxes):
        torch.save(input_boxes[i], prepared_pieces_dir / f"Inputs Box {i}" / f"{name}.pt",)
        torch.save(label_boxes[i], prepared_pieces_dir / f"Label Box {i}" / f"{name}.pt",)
    
    j+=1
    if j == number_datapoints:
        break

## Store boxes for 2 levels in 2 datasets

In [33]:
# prepare 1st level
prepared_dir_1stlevel = Path(paths["datasets_prepared_dir"]) / f"{dataset_name} cut_{number_boxes}pieces separate_boxes 1st level"
prepared_dir_1stlevel.mkdir(parents=True, exist_ok=True)

shutil.copy(prepared_pieces_dir / "info.yaml", prepared_dir_1stlevel / "info.yaml")
shutil.copytree(prepared_pieces_dir / "Inputs Box 0", prepared_dir_1stlevel / "Inputs")
shutil.copytree(prepared_pieces_dir / "Label Box 0", prepared_dir_1stlevel / "Labels")

PosixPath('/home/pillerls/heat-plume/datasets/prepared/benchmark_dataset_2d_100datapoints inputs_pksi cut_16pieces separate_boxes 1st level/Labels')

In [35]:
# prepare 2nd level
prepared_dir_2ndlevel = Path(paths["datasets_prepared_dir"]) / f"{dataset_name} cut_{number_boxes}pieces separate_boxes 2nd level gkt"
prepared_dir_2ndlevel.mkdir(parents=True, exist_ok=True)
(prepared_dir_2ndlevel / "Inputs").mkdir(parents=True, exist_ok=True)
(prepared_dir_2ndlevel / "Labels").mkdir(parents=True, exist_ok=True)

info = yaml.safe_load(open(prepared_dir_1stlevel / "info.yaml", "r"))
info_p    = deepcopy(info["Labels"]["Liquid Pressure [Pa]"])
info_k    = deepcopy(info["Inputs"]["Permeability X [m^2]"])
info_T    = deepcopy(info["Labels"]["Temperature [C]"])
info["Inputs"]["Temperature [C]"] = info_T
info["Inputs"]["Liquid Pressure [Pa]"] = info_p
info["Inputs"]["Permeability X [m^2]"] = info_k
info["Inputs"]["Permeability X [m^2]"]["index"] = 2
info["Inputs"]["Liquid Pressure [Pa]"]["index"] = 1
info["Inputs"]["Temperature [C]"]["index"] = 0
# assert indices of inputs double
idx_p = info["Inputs"]["Liquid Pressure [Pa]"]["index"]
idx_k = info["Inputs"]["Permeability X [m^2]"]["index"]
idx_t = info["Inputs"]["Temperature [C]"]["index"]
assert  idx_p != idx_k, "indices of inputs double"
assert  idx_p != idx_t, "indices of inputs double"
assert  idx_k != idx_t, "indices of inputs double"
info["Inputs"].pop("Material ID")
info["Inputs"].pop("SDF")
info["Labels"].pop("Liquid Pressure [Pa]")
print(info["Inputs"])

yaml.safe_dump(info, open(prepared_dir_2ndlevel / "info.yaml", "w"))

for box in range(2, number_boxes-2):
    for file_in_temp in (prepared_pieces_dir / f"Label Box {box}").iterdir():
        file_id = int(file_in_temp.stem.split("_")[1])
        new_id = file_id + (box) * 1000
        temp_in = torch.load(file_in_temp)[0]
        file_inputs = prepared_pieces_dir / f"Inputs Box {box}" / f"RUN_{file_id}.pt"
        file_label = prepared_pieces_dir / f"Label Box {box+1}" / f"RUN_{file_id}.pt"
        p_in = torch.load(file_label)[1]
        k_in = torch.load(file_inputs)[1]
        inputs = torch.zeros([3, *p_in.shape])
        inputs[idx_p] = p_in
        inputs[idx_k] = k_in
        inputs[idx_t] = temp_in[-1:,:].repeat(16, 1)
        
        torch.save(inputs, prepared_dir_2ndlevel / "Inputs" / f"RUN_{new_id}.pt")

        outputs = torch.zeros([1, *p_in.shape])
        outputs[0] = torch.load(file_label)[0]
        torch.save(outputs, prepared_dir_2ndlevel / "Labels" / f"RUN_{new_id}.pt")

        #print(inputs.shape, outputs.shape)

{'Liquid Pressure [Pa]': {'index': 1, 'max': 914955.125, 'mean': 895024.375, 'min': 866588.4375, 'norm': 'Rescale', 'std': 4318.6884765625}, 'Permeability X [m^2]': {'index': 2, 'max': 4.370007467002779e-09, 'mean': 6.180664313681916e-10, 'min': 1.0262329180887875e-11, 'norm': 'Rescale', 'std': 9.46576039773106e-10}, 'Temperature [C]': {'index': 0, 'max': 15.600923538208008, 'mean': 11.38478946685791, 'min': 10.60000228881836, 'norm': 'Rescale', 'std': 1.178992509841919}}
